# **Using a Sequential method to build a neural network..**

So far, we have built a neural network by defining a class where we define the layers and how the layers are connected with each other. In this section, we will learn about a simplified way of defining the neural network architecture using the Sequential class. We will perform the same steps as we have done in the previous sections, except that the class that was used to define the neural network architecture manually will be substituted with a Sequential class for creating a neural network architecture.

# **Define the toy dataset:**

In [1]:
x =  [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

# Import the relevant packages and define the device we will work on..

In [3]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset,DataLoader
device = 'cuda'if torch.cuda.is_available() else 'gpu'

# Now, we define the dataset class (MyDataset):

In [4]:
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)

    def __getitem__(self,ix):
        return self.x[ix],self.y[ix]

    def __len__(self):
        return len(self.x)


# Define the dataset (ds) and dataloader (dl) object:

In [8]:
ds = MyDataset(x,y)
dl = DataLoader(ds,batch_size=2,shuffle=True)

# Define the model architecture using the Sequential method available in the nn package:

In [23]:
model = nn.Sequential(
    nn.Linear(2, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
).to(device)

Note that, in the preceding code, we defined the same architecture of the network as we defined in previous sections, but defined differently. nn.Linear accepts two-dimensional input and gives an eight-dimensional output for each data point. Furthermore, nn.ReLU performs ReLU activation on top of the eight-dimensional output and finally, the eight-dimensional input gives a one-dimensional output (which in our case is the output of the addition of the two inputs) using the final nn.Linear layer.

# Install and import the package that enables us to print the model summary:

In [24]:
!pip install torch_summary
from torchsummary import summary

# Print a summary of the model, which expects the name of the model and also the input size of the model:

In [25]:
summary(model, torch.zeros(1,2));

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 8]                   24
├─ReLU: 1-2                              [-1, 8]                   --
├─Linear: 1-3                            [-1, 1]                   9
Total params: 33
Trainable params: 33
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


## Note that the output shape of the first layer is (-1, 8), where -1 represents that there can be as many data points as the batch size, and 8 represents that for each data point, we have an eight-dimensional output resulting in an output of the shape batch size x 8. The interpretation for the next two layers is similar.

# Next, we define the loss function (loss_func) and optimizer (opt) and train the model, just like we did in the previous section. Note that, in this case, we need not define a model object; a network is not defined within a class in this scenario:

In [26]:
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(model.parameters(),lr=0.001)
import time
loss_history = []
start =time.time()
for _ in range(50):
    for ix,iy in dl:
        opt.zero_grad()
        loss_value = loss_func(model(ix),iy)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)

end = time.time()
print(end-start)

0.19577646255493164


# Now that we have trained the model, we can predict values on a validation dataset that we define now:

# Define the validation dataset:

In [27]:
val = [[8,9],[10,11],[1.5,2.5]]

Predict the output of passing the validation list through the model (note that the expected value is the summation of the two inputs for each list within the list of lists). As defined in the dataset class, we first convert the list of lists into a float after converting them into a tensor object and registering them to the device:

In [28]:
model(torch.tensor(val).float().to(device))

tensor([[16.4989],
        [20.1449],
        [ 4.6493]], device='cuda:0', grad_fn=<AddmmBackward>)

Now that we have learned about leveraging the sequential method to define and train a model